In [1]:
import cv2
from cv2 import aruco
from matplotlib import pyplot as plt
import math
from math import atan2, cos, sin, sqrt, pi
from sympy import Point, Polygon, Line, Segment, plot, Ray
import numpy as np
from tdmclient import ClientAsync

In [2]:
def convertWheelSpeeds(speed_l,speed_r,speed_factor=3,axle_track=94):
    '''
    Function that takes the raw values of each motor angular speed and converts it into
    translation speed (in mm/s) and rotation speed (rad/sec) with a pixel scaling factor.

    Inputs: - speed_l: speed of left motor (no units, from Thymio measurements)
            - speed_r: speed of right motor (no units, from Thymio measurements)
            - speed_factor: 3 (motor_target of 200 sent to Thymio corresponds to a wheel speed of 66.8 mm/s)
            - axle_track: 94mm (distance between both wheels)

    Output: - v  : The current forward velocity of the Thymio (in mm/s)
            - omega : The current angular velocity of the Thymio (in rad/s)(positive direction corresponding to that of the camera frame)
    '''
    # wheel_radius : 22mm
    # axle track : 94mm (distance between the 2 wheels)
    v = 0.5*(speed_l + speed_r) / speed_factor
    omega = (speed_l - speed_r) / (0.5*axle_track*speed_factor)
    
    return v, omega
    

In [3]:
def px2real(x_px, scalingFactor):
    '''
    Function that converts measurement in pixels to measurement in mm.

    Inputs: - x_px : value in pixels
            - scalingFactor : for image size (1920x1080) corresponding to real length and width (800mm x 450mm), the scalingFactor is 2.4 

    Output: - x_real : value in mm
            
    '''
    x_real = x_px / 2.4
    return x_real
    
    

In [4]:
def real2px(x_real, scalingFactor):
    '''
    Function that converts measurement in mm to measurement in pixels.

    Inputs: - x_real : value in mm
            - scalingFactor : for image size (1920x1080) corresponding to real length and width (800mm x 450mm), the scalingFactor is 2.4 

    Output: - x_px : value in pixels
            
    '''
    x_px = x_real * 2.4
    return x_px
    

In [5]:
'''
Extended Kalman Filter class

Code modified from https://github.com/L42Project/Tutoriels/blob/master/Divers/tutoriel36/KalmanFilter.py
Comments describing Kalman Filter taken from https://stackoverflow.com/questions/74318200/how-to-tune-extended-kalman-filter-on-pykalman

Example of instance: KF=KalmanFilter(0.03333, [0, 0, 0, 0, 0]) (dt = 0.033 is the time between each measurement, x_ini = [0, 0, 0, 0, 0] is the initial state)

For update() function : takes as input the measurement vector z
z[0:2] are the measurements from the camera (px, py, orientation)
z[3:4] are the measurements from the Thymio (forward speed v and angular speed omega)

When Thymio position not found by camera -> z becomes 2x1 vector, with z[0:1] being the forward speed v and angular speed omega
'''


class KalmanFilter():
    def __init__(self, dt, x_ini):
        self.dt = dt

        # Initial State Vector
        self.x=np.matrix([[x_ini[0]], [x_ini[1]], [x_ini[2]], [x_ini[3]], [x_ini[4]]])
    

        # Initialize State Transition Matrix (Warning : changes with time -> updateFk())
        self.Fk=np.eye(5)

        # Initialize Measurement matrix H
        # Used to convert the predicted state estimate into predicted sensor measurements at time k.
        # In this case, H will be the identity matrix since the estimated state maps directly to state measurements data
        # H has the same number of rows as sensor measurements and same number of columns as states.
        self.H=np.eye(5)

        
        # State model noise covariance matrix Q
        # When Q is large, the Kalman Filter tracks large changes in
        # the sensor measurements more closely than for smaller Q.
        # Q is a square matrix that has the same number of rows as states.
        self.Q=np.diag([1, 0.1, 0.01, 1.0, 1.0]) 
        

        # Sensor measurement noise covariance matrix R
        # Has the same number of rows and columns as sensor measurements.
        # If we are sure about the measurements, R will be near zero.
        self.coord_var = 0.01
        self.angle_var = 0.0001
        self.speed_var = 6.0
        self.omega_var = 0
        
        self.R=np.diag([self.coord_var, self.coord_var, self.angle_var, self.speed_var, self.omega_var])

        self.P=np.eye(5)
        
    def updateFk(self):
        '''
        Function that updates the state transition matrix Fk
        '''
        
        self.Fk=np.matrix([[1.0, 0, 0, self.dt*math.cos(self.x[2]), 0],
                           [0, 1.0,  0, self.dt*math.sin(self.x[2]), 0],
                           [0, 0, 1.0, 0, self.dt],
                           [0, 0, 0, 1.0, 0],
                           [0, 0, 0, 0, 1.0]])
        self.Fk = self.Fk.astype(float)

    def predict(self):
        # Update Fk
        self.updateFk()
        # Predict the state estimate (A Priori) at time k based on the state estimate at time k-1
        self.x=np.dot(self.Fk, self.x)
        # Predict the state covariance estimate based on the previous covariance and some noise
        self.P=np.dot(np.dot(self.Fk, self.P), self.Fk.T)+self.Q
        return self.x

    def update(self, z, CameraAccessible = True): # z[0:2] corrpesonds to measurement of camera, z[3:4] corresponds to measurements of wheels
        
        if CameraAccessible:
            self.H=np.eye(5)
            
            self.R=np.diag([self.coord_var, self.coord_var, self.angle_var, self.speed_var, self.omega_var])
        else: # CAUTION : IF CAMERAACCESSIBLE = FALSE, Z IS A 2x1 VECTOR (z[0:1] are now measrurements from wheels)
            # Measurement matrix H (now only v and omega can be observed)
            self.H=np.matrix([[0, 0, 0, 1, 0],
                              [0, 0, 0, 0, 1]])
            
            self.R=np.diag([self.speed_var, self.omega_var])
            
            
        # Compute Kalman gain
        S=np.dot(self.H, np.dot(self.P, self.H.T))+self.R
        inv_S = np.linalg.pinv(S.astype(float))
        K=np.dot(np.dot(self.P, self.H.T),inv_S)

        # Correction / innovation
        # Calculate an updated state estimate (A Posteriori) for time k
        self.x=self.x+np.dot(K, (z-np.dot(self.H, self.x)))
        # Update the state covariance estimate for time k
        I=np.eye(self.H.shape[1])
        self.P=(I-(K*self.H))*self.P


        return self.x

In [6]:
def dist(pt1, pt2):
    distance = ( (pt1[0]-pt2[0])**2 + (pt1[1]-pt2[1])**2 )**0.5
    return distance

In [7]:
def get_thymio_position(img, img_output):
    
    '''
    Function that outputs Thymio position, radius and orientation from camera frames using Aruco marker detection
    Parts of code taken from https://mecaruco2.readthedocs.io/en/latest/notebooks_rst/Aruco/aruco_basics.html

    Inputs: - img : image from camera in BGR format
            - img_output : copy of image img on which will be drawn Thymio's contours

    Output: - center : tuple (x,y) representing the center of the Thymio (in pixels)
            - radius : radius of the Thymio (in pixels)
            - angle : orientation of the Thymio (in radians)
            - thymio_detected : boolean set to True if Thymio was detected, False if not
    '''
    
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    aruco_dict = aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
    parameters =  aruco.DetectorParameters_create()
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    frame_markers = aruco.drawDetectedMarkers(img_output, corners, ids)
    
    
    thymio_detected = False
    center = (0,0)
    radius = 0
    angle = 0
    
    if np.all(ids != None):
        for i in range(len(ids)):
            if ids[i] == 0:
                thymio_detected = True
                c = corners[i][0]

                top_center = (c[0:2, 0].mean(),c[0:2, 1].mean())
                bottom_center = (c[2:4, 0].mean(), c[2:4, 1].mean())
                left_center = (c[0:4:3, 0].mean(), c[0:4:3, 1].mean())
                right_center = (c[1:3, 0].mean(), c[1:3, 1].mean())
                points = np.array([ [c[2,0],c[2,1]], [c[3,0],c[3,1]], [left_center[0],left_center[1]], [right_center[0],right_center[1]] ])

                square_center = (int(c[:, 0].mean()), int(c[:, 1].mean()))
                radius = int(max( dist(square_center,c[0]), dist(square_center,c[1]), dist(square_center,c[2]), dist(square_center,c[3])))
                angle = math.atan2(top_center[1]-bottom_center[1],top_center[0]-bottom_center[0])
                
                
                center = (int(points[:, 0].mean()), int(points[:, 1].mean()))
                cv2.arrowedLine(img_output, (center[0],center[1]), (int(top_center[0]),int(top_center[1])), (255,255,0), 10)
                cv2.circle(img_output,square_center,radius,(255,0,0),10)

                break

        
    return center, radius, angle, thymio_detected

In [8]:
import matplotlib.pyplot as plt
from sympy import Point, Polygon, Line, Segment, plot, Ray
import random
import numpy as np
import time

# path = [[200, 1200], [765, 1100], [1176, 680], [1300, 400]]
# thymioVector = [[765,1100],[1300,1800]]
# center = Point(thymioVector[0])
# thymioAnglePoint = Point(thymioVector[1])
# thymioRay = Ray(center,thymioAnglePoint)
# print(path)

def make_ray_list(path):
    rays = {}
    for i in range(0, len(path)-1):
        rays[i] = Ray(path[i],path[i+1])
        print(rays)

    return rays

def find_error_angle(center,thymioAngle,rays,j):
    thymioRay = Ray(center,angle=thymioAngle)
    print(center)
    angle = (180/np.pi)*(rays[j].closing_angle(thymioRay)).evalf()
    if (angle > 180):
        angle = -1*(360 - angle)
    print(angle)
    return angle

def column(matrix, i):
    return [row[i] for row in matrix]

def plot_2D_array(array,color):
    plt.rcParams["figure.figsize"] = [7.50, 7.50]
    plt.rcParams["figure.autolayout"] = True
    xvalues = column(array,0)
    yvalues = column(array,1)
    plt.plot(xvalues,yvalues,color)
    plt.xlim(0, 2300)
    plt.ylim(0, 2300)
        
def orient(center,thymioAngle,path,kp):
    lineList = make_ray_list(path)  
    print(center,thymioAngle)
    e = find_error_angle(center,thymioAngle,lineList,0)
    a = -e*kp
    b = e*kp
    a = int(a)
    b = int(b)
    return a,b,e


def control(center,thymioAngle,path,baseSpeed,kp):
    lineList = make_ray_list(path)  
    e = find_error_angle(center,thymioAngle,lineList,0)
    print(e)
    if (e > 1):
        a = baseSpeed - kp*e
        b = baseSpeed
    if (e < -1):
        print('Its less than -1')
        a = baseSpeed
        b = baseSpeed + kp*e
    else: 
        a = baseSpeed
        b = baseSpeed
    a = int(a)
    b = int(b)
    return a, b, e

In [9]:
client = ClientAsync()
node = await client.lock()

In [10]:
'''
Testing Kalman Filter
'''
from sympy.solvers import linsolve
global center, radius, angle, thymio_detected

forward_speed = 0
angular_speed = 0

KF=KalmanFilter(0.033, [500, 300, math.pi/4, forward_speed, angular_speed])



### INITIALIZE CAMERA
cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
fps = int(cap.get(5))
print("fps:", fps)

success, img = cap.read()
imgResult = img.copy()
    
thymioLoc, thymioRadius, thymioAngle, thymioDetected = get_thymio_position(img, imgResult)


path = [[thymioLoc[0],thymioLoc[1]],[thymioLoc[0]+300,thymioLoc[1]+300],[thymioLoc[0]+500,thymioLoc[1]-300]]

### MAIN LOOP
count = 0

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

    
# running infinite while loop so that
# program keep running until we close it
# previous to the main navigation loop, run orientation loop:
while True:
    success, img = cap.read()
    imgResult = img.copy()
    
    thymioLoc, thymioRadius, thymioAngle, thymioDetected = get_thymio_position(img, imgResult)
    
    # KALMAN -------------------------------------------------------------------
    KF.predict()
    if thymioDetected:
        z = np.array([thymioLoc[0],thymioLoc[1],thymioAngle, forward_speed, angular_speed])
        z = np.expand_dims(z, axis=-1)
    else:
        z = np.array([forward_speed, angular_speed]) 
        z = np.expand_dims(z, axis=-1)
    KF.update(z,thymioDetected)
    cv2.circle(imgResult, (int(KF.x[0]), int(KF.x[1])), 10, (255,0,255), 10)
    # KALMAN ----------------------------------------------------------------------
    count = count + 1
    #print(count)
    #print('Thymio center:',thymioLoc, '  Angle:',thymioAngle, '  Thymio Detected:' ,thymioDetected)
    #print('Kalman center:',KF.x[0:2],'  Kalman angle:',KF.x[2], 'Kalman speed and omega', KF.x[3:5])
    
    #Path tracking ----------------------------------------------------------------
    x = KF.x[0].tolist()[0][0]
    y = KF.x[1].tolist()[0][0]
    print(x)
    print(y)
    print(type(x))
    center = Point(x,y)
    angle = KF.x[2].tolist()[0][0]
    print(angle)

    kp = 0.1

    [leftSpeed,rightSpeed,error] = orient(center,angle,path,kp)
    await node.set_variables({'motor.left.target':[leftSpeed]}) 
    await node.set_variables({'motor.right.target':[rightSpeed]}) 
    time.sleep(0.2)

    # displaying output on Screen
    imgResult = cv2.line(imgResult,tuple(path[0]),tuple(path[1]),(255,255,255),1)
    imgResult = cv2.line(imgResult,tuple(path[1]),tuple(path[2]),(255,255,255),1)
    cv2.imshow("Result", imgResult)
    
    c = cv2.waitKey(1)
    if (abs(error)<10):
        await node.set_variables({'motor.left.target':[0]}) 
        await node.set_variables({'motor.right.target':[0]}) 
        break


while True:
    success, img = cap.read()
    imgResult = img.copy()
    
    thymioLoc, thymioRadius, thymioAngle, thymioDetected = get_thymio_position(img, imgResult)
    
    # KALMAN -------------------------------------------------------------------
    KF.predict()
    if thymioDetected:
        z = np.array([thymioLoc[0],thymioLoc[1],thymioAngle, forward_speed, angular_speed])
        z = np.expand_dims(z, axis=-1)
    else:
        z = np.array([forward_speed, angular_speed]) 
        z = np.expand_dims(z, axis=-1)
    KF.update(z,thymioDetected)
    cv2.circle(imgResult, (int(KF.x[0]), int(KF.x[1])), 10, (255,0,255), 10)
    # KALMAN ----------------------------------------------------------------------
    count = count + 1
    #print(count)
    #print('Thymio center:',thymioLoc, '  Angle:',thymioAngle, '  Thymio Detected:' ,thymioDetected)
    #print('Kalman center:',KF.x[0:2],'  Kalman angle:',KF.x[2], 'Kalman speed and omega', KF.x[3:5])
    
    #Path tracking ----------------------------------------------------------------
    x = KF.x[0].tolist()[0][0]
    y = KF.x[1].tolist()[0][0]
    print(x)
    print(y)
    print(type(x))
    center = Point(x,y)
    angle = KF.x[2].tolist()[0][0]
    baseSpeed = 20

    [leftSpeed,rightSpeed,error] = control(center,angle,path,baseSpeed,0.2)
    await node.set_variables({'motor.left.target':[leftSpeed]}) 
    await node.set_variables({'motor.right.target':[rightSpeed]}) 
    time.sleep(0.2)

    # displaying output on Screen
    imgResult = cv2.line(imgResult,tuple(path[0]),tuple(path[1]),(255,255,255),1)
    imgResult = cv2.line(imgResult,tuple(path[1]),tuple(path[2]),(255,255,255),1)
    cv2.imshow("Result", imgResult)
    
    c = cv2.waitKey(1)
    if c == 27:
        break

fps: 30
718.9059879857242
418.9198521182932
<class 'float'>
-2.22494301816809
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(179726496996431/250000000000, 418919852118293/1000000000000) -2.22494301816809
Point2D(179726496996431/250000000000, 418919852118293/1000000000000)
-187.520155401869
720.9788708408657
418.9866733665957
<class 'float'>
-2.2117110605818033
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(360489435420433/500000000000, 104746668341649/250000000000) -2.2117110605818033
Point2D(360489435420433/500000000000, 104746668341649/250000000000)
-188.278290726260
720.0091948878671
418.9943100668043
<class 'float'>
-2.1366886352017613
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), P

722.9999624166904
422.00003894082926
<class 'float'>
-0.09328369819477206
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(72299996241669/100000000000, 422000038940829/1000000000000) -0.09328369819477206
Point2D(72299996241669/100000000000, 422000038940829/1000000000000)
-309.655237796067
722.0097481035455
422.91613113164044
<class 'float'>
0.026775911553920034
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(361004874051773/500000000000, 10572903278291/25000000000) 0.026775911553920034
Point2D(361004874051773/500000000000, 10572903278291/25000000000)
43.4658532753448
722.0000581386529
422.99295255496327
<class 'float'>
0.11136521582088772
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point

725.0000112198027
422.07742094079686
<class 'float'>
0.07581340435544998
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(725000011219803/1000000000000, 422077420940797/1000000000000) 0.07581340435544998
Point2D(725000011219803/1000000000000, 422077420940797/1000000000000)
40.6562118999140
724.0098061772771
422.92257788210446
<class 'float'>
0.13833963557150306
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(724009806177277/1000000000000, 52865322235263/125000000000) 0.13833963557150306
Point2D(724009806177277/1000000000000, 52865322235263/125000000000)
37.0737227423750
724.000098348065
422.9935053730813
<class 'float'>
0.1389526967695204
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point

727.0000062186177
423.00000139756753
<class 'float'>
0.00020911523568172344
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(363500003109309/500000000000, 6609375021837/15625000000) 0.00020911523568172344
Point2D(363500003109309/500000000000, 6609375021837/15625000000)
44.9880185795636
727.9902157339377
423.0000003277462
<class 'float'>
-0.013943437929805923
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(363995107866969/500000000000, 211500000163873/500000000000) -0.013943437929805923
Point2D(363995107866969/500000000000, 211500000163873/500000000000)
-314.201099854720
727.9999178075399
422.99999838182015
<class 'float'>
-0.014082201789867436
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), 

730.0000067848005
423.00000329654057
<class 'float'>
0.0420065560587633
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(912500008481/1250000000, 423000003296541/1000000000000) 0.0420065560587633
Point2D(912500008481/1250000000, 423000003296541/1000000000000)
42.5932016259532
730.990216081501
423.00000813680845
<class 'float'>
0.007384958483483119
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(730990216081501/1000000000000, 52875001017101/125000000000) 0.007384958483483119
Point2D(730990216081501/1000000000000, 52875001017101/125000000000)
44.5768730470171
730.999918052116
423.00000148612367
<class 'float'>
-0.006851920125835899
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 

...

964.9807395218558
618.8207447946129
<class 'float'>
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(15077824055029/15625000000, 618820744794613/1000000000000)
3.23126375035159
3.23126375035159
966.9807331385927
620.8212128487955
<class 'float'>
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(966980733138593/1000000000000, 155205303212199/250000000000)
2.95352922278906
2.95352922278906
968.9807280919314
622.8212542957328
<class 'float'>
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(968980728091931/1000000000000, 622821254295733/1000000000000)
2.95080607349820
2.95080607349820
970.9807262861891
624.8212450649568
<class 'float'>
{0: Ray2D(Point2D(721, 419), Point

1024.9709065826241
672.8980694447956
<class 'float'>
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(51248545329131/50000000000, 168224517361199/250000000000)
2.04072834162730
2.04072834162730
1026.9806830128705
675.7443563969198
<class 'float'>
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(102698068301287/100000000000, 16893608909923/25000000000)
2.61011681842013
2.61011681842013
1028.9807685953106
677.815173290625
<class 'float'>
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(102898076859531/100000000000, 216900855453/320000000)
2.36381354124583
2.36381354124583
1031.9709566016516
679.8210942685682
<class 'float'>
{0: Ray2D(Point2D(721, 419), Point2D(1021, 

1084.9807320394466
731.8216785192897
<class 'float'>
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(21699614640789/20000000000, 73182167851929/100000000000)
0.860838197791402
0.860838197791402
1086.9807293378483
733.8216151681155
<class 'float'>
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(21739614586757/20000000000, 146764323033623/200000000000)
0.575761513148480
0.575761513148480
1089.9709279522776
735.8216870943016
<class 'float'>
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(27249273198807/25000000000, 367910843547151/500000000000)
0.860866944486222
0.860866944486222
1091.9806344102863
737.8216257262625
<class 'float'>
{0: Ray2D(Point2D(721, 419), Poin

1143.9807838533964
791.8160106228598
<class 'float'>
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(5719903919267/5000000000, 39590800531143/50000000000)
-3.96462883141761
-3.96462883141761
Its less than -1
1146.970960745486
793.8223003542785
<class 'float'>
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(114697096074549/100000000000, 396911150177139/500000000000)
-4.00378521609293
-4.00378521609293
Its less than -1
1147.990446989462
795.8225628449956
<class 'float'>
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719))}
{0: Ray2D(Point2D(721, 419), Point2D(1021, 719)), 1: Ray2D(Point2D(1021, 719), Point2D(1221, 119))}
Point2D(57399522349473/50000000000, 198955640711249/250000000000)
-3.92667128210735
-3.92667128210735
Its less than -1
1149.9808230645715
797.8224772957493
<

CancelledError: 